In [16]:
import pandas as pd
import math

In [4]:
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')

In [5]:
print("Length of train data and test data: %s and %s, respectively"
     % (len(train_data), len(test_data)))

Length of train data and test data: 17384 and 4229, respectively


In [17]:
def add_additional_columns(data_set):
    data_set['bedrooms_squared'] = data_set['bedrooms'].apply(lambda u: u*u)
    data_set['bed_bath_rooms'] = data_set['bedrooms'] * data_set['bathrooms']
    data_set['log_sqft_living'] = data_set['sqft_living'].apply(lambda u: math.log(u))
    data_set['lat_plus_long'] = data_set['lat'] + data_set['long']

In [18]:
add_additional_columns(train_data)

train_data.head()

In [20]:
add_additional_columns(test_data)
test_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,114101516,20140528T000000,310000.0,3,1.0,1430,19901,1.5,0,0,...,0,98028,47.7558,-122.229,1780,12697,9,3.0,7.265430,-74.4732
1,9297300055,20150124T000000,650000.0,4,3.0,2950,5000,2.0,0,3,...,0,98126,47.5714,-122.375,2140,4000,16,12.0,7.989560,-74.8036
2,1202000200,20141103T000000,233000.0,3,2.0,1710,4697,1.5,0,0,...,0,98002,47.3048,-122.218,1030,4705,9,6.0,7.444249,-74.9132
3,8562750320,20141110T000000,580500.0,3,2.5,2320,3980,2.0,0,0,...,0,98027,47.5391,-122.070,2580,3980,9,7.5,7.749322,-74.5309
4,7589200193,20141110T000000,535000.0,3,1.0,1090,3000,1.5,0,0,...,0,98117,47.6889,-122.375,1570,5080,9,3.0,6.993933,-74.6861


### Quiz question: what are the mean values of your 4 new variables on TEST data? (round to 2 digits)

In [23]:
def show_mean(data_set, columns):
    for col in columns:
        col_mean = data_set[col].mean()
        print ('Mean of col: %s is %s' % (col, col_mean))

In [24]:
show_mean(test_data, ['bedrooms_squared', 'bed_bath_rooms',
                         'log_sqft_living', 'lat_plus_long'
                     ])

Mean of col: bedrooms_squared is 12.4466777015843
Mean of col: bed_bath_rooms is 7.5039016315913925
Mean of col: log_sqft_living is 7.550274679645938
Mean of col: lat_plus_long is -74.65333355403168


In [25]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

### Model 1

In [94]:
regr = linear_model.LinearRegression()

In [95]:
input_1 = pd.DataFrame(train_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long']])

In [96]:
print(regr.fit(input_1, train_data['price']).coef_)

[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]


In [97]:
coef_1 = regr.coef_
print(coef_1)

[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]


### Model 2

In [98]:
input_2 = pd.DataFrame(train_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long',
                                   'bed_bath_rooms'
                                  ]])

In [99]:
regr2 = linear_model.LinearRegression()

In [100]:
regr2.fit(input_2, train_data['price'])
coef_2 = regr2.coef_
print(coef_2)

[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04]


### Model 3

In [101]:
input_3 = pd.DataFrame(train_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long',
                                   'bed_bath_rooms',
                                   'bedrooms_squared',
                                   'log_sqft_living',
                                   'lat_plus_long'
                                  ]])

In [102]:
regr3 = linear_model.LinearRegression()

In [103]:
regr3.fit(input_3, train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [104]:
coef_3 = regr3.coef_
print(coef_3)

[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
  1.27334900e+05]


### Quiz Question: Which model (1, 2 or 3) had the lowest RSS on TRAINING data?

In [105]:
def calculate_rss(model, input_x, actual_output):
    diff = model.predict(input_x) - actual_output
    return diff.transpose().dot(diff)

In [109]:
rss_1 = calculate_rss(regr, input_1, train_data['price'])
rss_2 = calculate_rss(regr2, input_2, train_data['price'])
rss_3 = calculate_rss(regr3, input_3, train_data['price'])
print("Rss1 = %s, Rss2 = %s, Rss3 = %s" % (rss_1, rss_2, rss_3))
print("Min is: %s, Max is: %s " % (min(rss_1, rss_2, rss_3),
                                max(rss_1, rss_2, rss_3)
                               ))

Rss1 = 967879963049545.8, Rss2 = 958419635074069.0, Rss3 = 903436455050477.9
Min is: 903436455050477.9, Max is: 967879963049545.8 


### Quiz Question: Which model (1, 2, or 3) had the lowest RSS on TESTING data?

In [118]:
input_test_1 = pd.DataFrame(test_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long']])
input_test_2 = pd.DataFrame(test_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long',
                                   'bed_bath_rooms'
                                  ]])

input_test_3 = pd.DataFrame(test_data[['sqft_living', 
                                   'bedrooms',
                                   'bathrooms',
                                   'lat',
                                   'long',
                                   'bed_bath_rooms',
                                   'bedrooms_squared',
                                   'log_sqft_living',
                                   'lat_plus_long'
                                  ]])


In [119]:
rss_test_1 = calculate_rss(regr, input_test_1, test_data['price'])
rss_test_2 = calculate_rss(regr2, input_test_2, test_data['price'])
rss_test_3 = calculate_rss(regr3, input_test_3, test_data['price'])

print("Analysis on test data")
print("Rss1 = %s, Rss2 = %s, Rss3 = %s" % (rss_test_1, rss_test_2, rss_test_3))
print("Min is: %s, Max is: %s " % (min(rss_test_1, rss_test_2, rss_test_3),
                                max(rss_test_1, rss_test_2, rss_test_3)
                               ))

Analysis on test data
Rss1 = 225500469795489.6, Rss2 = 223377462976467.2, Rss3 = 259236319207179.34
Min is: 223377462976467.2, Max is: 259236319207179.34 
